# XResNet.

> xResNet model.

In [ ]:
# hide
import torch

from functools import partial

from model_constructor import ModelConstructor
from model_constructor.xresnet import xresnet_stem

## XResNet constructor.

In [ ]:
xresnet = ModelConstructor(
    name="XResNet",
    make_stem=xresnet_stem,
    stem_sizes=[3, 32, 64, 64],
    act_fn=torch.nn.Mish,
)

In [ ]:
xresnet

XResNet
  in_chans: 3, num_classes: 1000
  block: BasicBlock,
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: Mish, sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
xresnet.print_changed_fields()

Changed fields:
name: XResNet
act_fn: Mish
stem_sizes: [3, 32, 64, 64]
make_stem: xresnet_stem


In [ ]:
# collapse_output
xresnet.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (conv_3): ConvBnAct(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_

In [ ]:
# collapse_output
xresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish(inplace=True)
    )
    (bl_1): BasicBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv)

In [ ]:
# collapse_output
xresnet.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

## xResNet constructors

Lets create constructor class for xResnet.

In [ ]:
# hide
from typing import Any, Callable, Optional
from model_constructor.helpers import ModSeq, MakeModule, nnModule
from torch import nn
from model_constructor.model_constructor import ModelCfg
from model_constructor.blocks import BottleneckBlock

In [ ]:
class McXResNet(ModelConstructor):
    make_stem: MakeModule = xresnet_stem
    stem_sizes: list[int] = [32, 32, 64]
    pool: Optional[nnModule] = partial(
        nn.AvgPool2d, kernel_size=2, ceil_mode=True
    )

xResnet34 inherit from xResnet.

In [ ]:
class McXResNet34(McXResNet):
    layers: list[int] = [3, 4, 6, 3]

xResnet50 inherit from xResnet34.

In [ ]:
class McXResNet50(McXResNet34):
    block: nnModule = BottleneckBlock
    block_sizes: list[int] = [256, 512, 1024, 2048]

Now we can create constructor from class adn change model parameters during initialization or after.

In [ ]:
mc = McXResNet34(num_classes=10)
mc

McXResNet34
  in_chans: 3, num_classes: 10
  block: BasicBlock,
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: ReLU, sa: False, se: False
  stem sizes: [32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

In [ ]:
mc = McXResNet50()
mc

McXResNet50
  in_chans: 3, num_classes: 1000
  block: BottleneckBlock,
  expansion: 1, groups: 1, dw: False, div_groups: None
  act_fn: ReLU, sa: False, se: False
  stem sizes: [32, 32, 64], stride on 0
  body sizes [256, 512, 1024, 2048]
  layers: [3, 4, 6, 3]

To create model - call model constructor object.

In [ ]:
model = mc()

In [ ]:
# hide
model

XResNet50(
  (stem): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): BottleneckBlock(
        (convs): Sequential(
          (conv_0

model_constructor
by ayasyrev